In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset=pd.read_csv("/content/drive/My Drive/DataLatihBDC_edited_3.csv")
dataset['judul']=dataset['judul'].astype(str)
dataset['narasi']=dataset['narasi'].astype(str)

In [ ]:
corpus = []
for i in range(0,len(dataset)):
    corpus.append(dataset.loc[i,'judul']+' '+dataset.loc[i,'narasi']) #ini ya ' '

In [ ]:
# Membuat model Bag of Words 
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X1 = cv.fit_transform(corpus)
y1 = dataset.loc[:, 'label'].values
newDataset = pd.DataFrame.sparse.from_spmatrix(X1,columns=cv.get_feature_names())
newDataset['label'] = y1

In [ ]:
#newDataset.to_csv("/content/drive/My Drive/datalatih(sebelum).csv",index=False,header=True)
print(newDataset.shape)

(4231, 14188)


In [ ]:
dataset_soal = pd.read_csv("/content/drive/My Drive/DataTestBDC_edited.csv")
dataset_soal['judul']=dataset_soal['judul'].astype(str)
dataset_soal['narasi']=dataset_soal['narasi'].astype(str)

In [ ]:
corpus_2 = []
for i in range(0,len(dataset_soal)):
    corpus_2.append(dataset_soal.loc[i,'judul']+' '+dataset_soal.loc[i,'narasi']) #ini ya ' '

In [ ]:
# Membuat model Bag of Words
X2 = cv.fit_transform(corpus_2)
#N = dataset.loc[:, 'label'].values
newDataset_soal = pd.DataFrame.sparse.from_spmatrix(X2, columns=cv.get_feature_names())
#newDataset_2 ['label'] = N

In [ ]:
newDataset = newDataset.filter([kata for kata in newDataset_soal.columns])
newDataset_soal = newDataset_soal.filter([kata2 for kata2 in newDataset.columns])
newDataset_soal.drop(columns=['label'],inplace=True)

In [ ]:
#newDataset.to_csv("/content/drive/My Drive/datalatih(sesudah).csv",index=False,header=True)

In [ ]:
"""a = pd.read_csv("/content/drive/My Drive/datalatih(sebelum).csv")
b = pd.read_csv("/content/drive/My Drive/datalatih(sesudah).csv")
print(a.shape)
print(b.shape)
"""

'a = pd.read_csv("/content/drive/My Drive/datalatih(sebelum).csv")\nb = pd.read_csv("/content/drive/My Drive/datalatih(sesudah).csv")\nprint(a.shape)\nprint(b.shape)\n'

In [ ]:
print(newDataset.columns)
print(newDataset_soal.columns)

Index(['aaj', 'aamiin', 'abai', 'abal', 'abdul', 'abis', 'aborsi', 'about',
       'abu', 'ac',
       ...
       'yusril', 'yuuu', 'yy', 'zaman', 'zat', 'zealand', 'zebra', 'zon',
       'zona', 'zuckerberg'],
      dtype='object', length=2862)
Index(['aaj', 'aamiin', 'abai', 'abal', 'abdul', 'abis', 'aborsi', 'about',
       'abu', 'ac',
       ...
       'yusril', 'yuuu', 'yy', 'zaman', 'zat', 'zealand', 'zebra', 'zon',
       'zona', 'zuckerberg'],
      dtype='object', length=2861)


In [ ]:
from imblearn.over_sampling import SMOTE
# for reproducibility purposes
seed = 1
# SMOTE number of neighbors
k = 5

# make a new df made of all the columns, except the target class
X_res = newDataset.loc[:, newDataset.columns != 'label']
y_res = newDataset.label
#sm = SMOTE(k_neighbors=k, random_state=seed)
#X_res, y_res = sm.fit_resample(X, y)

newDataset_res = pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res)], axis=1)
newDataset_res.columns = [i for i in range(0,2862)]
newDataset_res.rename(columns={ newDataset_res.columns[-1]: "label" }, inplace = True)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
newDataset_soal.columns = [i for i in range(0,2861)]
newDataset_soal.columns

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2851, 2852, 2853, 2854, 2855, 2856, 2857, 2858, 2859, 2860],
           dtype='int64', length=2861)

In [ ]:
# Membagi dataset ke dalam Training dan Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size = 0.2, random_state = 0)

In [ ]:
# Membuat model Random Forest Classification terhadap Training set
from sklearn.ensemble import RandomForestClassifier
classifierRF = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 0)
classifierRF.fit(X_train, y_train)

# Memprediksi hasil test set
y_pred_RF = classifierRF.predict(X_test)

# Membuat confusion matrix
#from sklearn.metrics import confusion_matrix
#cm_RF = confusion_matrix(y_test, y_pred_RF)

from sklearn.metrics import classification_report
print (classification_report(y_test, y_pred_RF))

              precision    recall  f1-score   support

           0       0.80      0.33      0.47       159
           1       0.86      0.98      0.92       688

    accuracy                           0.86       847
   macro avg       0.83      0.66      0.70       847
weighted avg       0.85      0.86      0.83       847



In [ ]:
y_pred_soal = classifierRF.predict(newDataset_soal)
y_pred_soal

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,

In [ ]:
data_test_kotor = pd.read_csv("/content/drive/My Drive/DataUjiBDC.tsv",sep='\t')

In [ ]:
y_pred_soal_csv = pd.concat([data_test_kotor['ID'],pd.DataFrame(y_pred_soal)],axis=1)
y_pred_soal_csv.columns = ['ID','prediksi']

In [ ]:
print(y_pred_soal_csv.head(10))
print(y_pred_soal_csv['prediksi'].value_counts())
print(y_pred_soal_csv.shape)

       ID  prediksi
0  238057         1
1  238158         1
2  238865         1
3  248298         1
4  255176         1
5  267051         1
6  269336         1
7  272438         0
8  282152         1
9  288490         1
1    428
0     42
Name: prediksi, dtype: int64
(470, 2)


In [ ]:
cek_hasil = pd.read_csv("/content/drive/My Drive/Datatest.tsv",sep='\t')
cek_hasil = cek_hasil.drop(columns = ['tanggal','nama file gambar','judul','narasi'])

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(cek_hasil['newlabel'], y_pred_soal_csv['prediksi']))

              precision    recall  f1-score   support

           0       0.57      0.37      0.45        65
           1       0.90      0.96      0.93       405

    accuracy                           0.87       470
   macro avg       0.74      0.66      0.69       470
weighted avg       0.86      0.87      0.86       470



In [ ]:
from sklearn.metrics import confusion_matrix
cm_RF = confusion_matrix(cek_hasil['label'], y_pred_soal_csv['prediksi'])
cm_RF

array([[ 26,  39],
       [ 16, 389]])

In [ ]:
precision = cm_RF[1][1]/(cm_RF[1][1]+cm_RF[1][0])
recall = cm_RF[1][1]/(cm_RF[1][1]+cm_RF[0][1])
f1 = 2*recall*precision/(recall+precision)
f1

0.9339735894357744

In [ ]:
#y_pred_soal_csv.to_csv("/content/drive/My Drive/HasilPrediksi.csv",index=False,header=True)

In [ ]:
sama = 0
beda = 0
for i in range(0,len(cek_hasil)):
  if (cek_hasil['label'][i] == y_pred_soal_csv['prediksi'][i]) :
    sama+=1
  else :
    beda+=1
print("yang bener : ",sama, " (",sama/len(cek_hasil)*100, "%)")
print("yang salah : ",beda, " (",beda/len(cek_hasil)*100, "%)")

yang bener :  415  ( 88.29787234042553 %)
yang salah :  55  ( 11.702127659574469 %)
